In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(777) # for reproductibility

2.1.0


In [11]:
x_data = [[1,2,1,1],
         [2,1,3,2],
         [3,1,3,4],
         [4,1,5,5],
         [1,7,5,5],
         [1,2,5,6],
         [1,6,6,6],
         [1,7,7,7]]
y_data = [[0,0,1],
         [0,0,1],
         [0,0,1],
         [0,1,0],
         [0,1,0],
         [0,1,0],
         [1,0,0],
         [1,0,0]]

# convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [9]:
nb_classes = 3 # class의 갯수

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [12]:
# Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W,b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.4564867 ,  0.53983474, -1.1366715 ],
       [ 0.51545775, -0.44776005, -0.72691107],
       [-0.44219774, -1.5832956 , -0.31117675],
       [-0.46808162,  1.0022584 , -0.55059594]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-0.19601157,  0.21892233, -0.69331926], dtype=float32)>


In [14]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[3.0027157e-01 6.7773020e-01 2.1998271e-02]
 [4.8193250e-02 9.3165678e-01 2.0150051e-02]
 [3.7100344e-04 9.9944788e-01 1.8108847e-04]
 [1.1353292e-04 9.9979514e-01 9.1305468e-05]
 [9.3598300e-01 6.3850135e-02 1.6696952e-04]
 [2.6503863e-02 9.7132540e-01 2.1707639e-03]
 [8.0059057e-01 1.9889010e-01 5.1927834e-04]
 [8.8340557e-01 1.1642074e-01 1.7364482e-04]], shape=(8, 3), dtype=float32)


In [15]:
# softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[4.5902415e-09 1.0000000e+00 2.4629851e-09]], shape=(1, 3), dtype=float32)


In [16]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(2.4330945, shape=(), dtype=float32)


In [17]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [18]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        
        return grads
    
print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.13058853,  0.6111318 , -0.7417203 ],
       [ 0.65518653, -0.16446908, -0.49071747],
       [ 0.40579358,  0.45677286, -0.8625665 ],
       [ 0.40312877,  0.46166247, -0.8647913 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.12442906,  0.24488954, -0.3693186 ], dtype=float32)>]


In [19]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' % (i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 2.031811
Loss at epoch 100: 0.649606
Loss at epoch 200: 0.551993
Loss at epoch 300: 0.493788
Loss at epoch 400: 0.447853
Loss at epoch 500: 0.407455
Loss at epoch 600: 0.369700
Loss at epoch 700: 0.332856
Loss at epoch 800: 0.295840
Loss at epoch 900: 0.260075
Loss at epoch 1000: 0.239424
Loss at epoch 1100: 0.227316
Loss at epoch 1200: 0.216379
Loss at epoch 1300: 0.206424
Loss at epoch 1400: 0.197322
Loss at epoch 1500: 0.188967
Loss at epoch 1600: 0.181268
Loss at epoch 1700: 0.174153
Loss at epoch 1800: 0.167558
Loss at epoch 1900: 0.161429
Loss at epoch 2000: 0.155718


#### prediction check

In [20]:
sample_data = [[2,1,3,2]] # answer_label = [0,0,1]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a,1)) # index: 2

tf.Tensor([[0.00205497 0.08289105 0.91505396]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [21]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[3.7569955e-06 1.1868312e-03 9.9880946e-01]
 [2.0549716e-03 8.2891054e-02 9.1505396e-01]
 [5.6705659e-08 1.6364072e-01 8.3635926e-01]
 [1.4599668e-06 8.4985423e-01 1.5014429e-01]
 [2.5336072e-01 7.3442239e-01 1.2216868e-02]
 [1.3299116e-01 8.6698902e-01 1.9764271e-05]
 [7.5716817e-01 2.4280141e-01 3.0437392e-05]
 [9.1867179e-01 8.1327625e-02 5.3354438e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


#### convert as class

In [27]:
class softmax_classifier(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifier, self).__init__()
        self.W = tf.Variable(tf.compat.v1.random_normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.compat.v1.random_normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            return grads
        
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
        
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' % (i+1, self.cost_fn(X,Y).numpy()))
                
model = softmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.583602
Loss at epoch 500: 0.340002
Loss at epoch 1000: 0.217327
Loss at epoch 1500: 0.174352
Loss at epoch 2000: 0.145320
